In [86]:
from qiskit import IBMQ, QuantumCircuit, transpile, assemble, execute

In [87]:
# Load account and set the simulator and desired real system
IBMQ.load_account()
provider = IBMQ.get_provider('ibm-q')

real = provider.get_backend('ibmq_belem')
sim = provider.get_backend('ibmq_qasm_simulator')

In [95]:
# Set Backend to either simulator or the real system.
backend = real
# backend = sim

# Set number of qubits for the selected system
num_qubits = backend.configuration().num_qubits

In [96]:
def create_rng_circuit(num_qubits):
    '''Creates circuit with Hadamard gates on each qubit from 0-num_qubits'''
    qc = QuantumCircuit(num_qubits, num_qubits)
    qc.h(range(num_qubits))
    qc.measure(range(num_qubits), range(num_qubits))
    return qc

# Uncertain with this one. I took what the system transpiles HGate into and plugged it directly into the circuit.
def create_rng_circuit_basis_gates(num_qubits):
    '''Creates circuit with basis gates representing a Hadamard gate on each qubit from 0-num_qubits'''
    qc = QuantumCircuit(num_qubits, num_qubits)
    qc.rz(1.5707963267948961, range(num_qubits))
    qc.sx(range(num_qubits))
    qc.rz(1.5707963267948966, range(num_qubits))
    qc.measure(range(num_qubits), range(num_qubits))
    return qc
    

def run(backend):
    '''Run the job on the given backend. Sends 75 copies of an rng circuit to the backend in one job.'''
    num_qubits = backend.configuration().num_qubits
    max_circs = 75 # Manually set to 75 since that is the max number of circuits per job for devices
    circ = create_rng_circuit(num_qubits)
    circ_list = [circ] * max_circs
    job = execute(circ_list, backend, shots=8192, memory=True)
    return job

def run_without_transpile(backend):
    '''Run the job on the given backend. Sends 75 copies of an rng circuit to the backend in one job.
       Skips the transpilation phase by creating a circuit with only basis gates.
    '''
    num_qubits = backend.configuration().num_qubits
    max_circs = 75 # Manually set to 75 since that is the max number of circuits per job for devices
    circ = create_rng_circuit_basis_gates(num_qubits)
    circ_list = [circ] * max_circs
    qobj = assemble(circ_list, shots=8192, memory=True)
    job = backend.run(qobj)
    return job

In [97]:
# Run the job, either with transpilation or without.
job = run(backend)
# job = run_without_transpile(backend)


In [98]:
# Create a list of outputs from each shot for each circuit
memory = []
for circ in range(75):
    memory += job.result().get_memory(circ)

In [99]:
def create_qubit_dict(num_qubits):
    '''Create a dictionary which has the following structure:
            {
            qubit 0: {'0': 0, '1': 0},
            ...
            qubit n: {'0': 0, '1': 0}
            }
    '''
    qubit_dict = {}
    for x in range(num_qubits):
        qubit_dict[x] = {'0': 0, '1': 0}
    return qubit_dict

In [100]:
# Populate qubit dictionary with all outputs in memory

# Create dict
per_qubit_output = create_qubit_dict(num_qubits)

# Set max qubit index
max_qubit = num_qubits - 1

# Populate dict
for output in memory:
    for idx, val in enumerate(output):
        qubit = max_qubit - idx # Since Qiskit's output has the LSB set to the right-most bit, we need to edit the idx
        per_qubit_output[qubit][val] = per_qubit_output[qubit].get(val, 0) + 1

In [101]:
# Print out the collected results

from pprint import pprint
pprint(per_qubit_output)

{0: {'0': 303830, '1': 310570},
 1: {'0': 317424, '1': 296976},
 2: {'0': 322724, '1': 291676},
 3: {'0': 318228, '1': 296172},
 4: {'0': 296809, '1': 317591}}
